# Explorando convolução no Keras usando o Theano

O objetivo deste notebook é de explorar a convolução que é implementada pelo Theano através do Keras.

A convolução é a operação essencial das redes convolucionais. É uma operação demorada e a sua implementação na GPU acelera bastante o seu processamento. A implementação é feita no Theano utilizando GPU. Se a GPU não for encontrada, a implementação roda na CPU.



In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.utils.np_utils import convert_kernel



Using Theano backend.


# Uma rede com uma única camada convolucional

O objetivo deste experimento é explorar numericamente a convolução implementada na rede
convolucional do Keras/Theano. Para isso iremos criar uma rede convolucional com apenas
uma camada de convolução. A sua predição será o resultado da rede, que neste caso será apenas uma convolução.

Uma convolução pode ser visto como uma média móvel ponderada sobre a imagem. A equação matemática da convolução é dada por:

$$ y = x \ast w $$

onde $x$ é a imagem de entrada, $y$ é o resultado da convolução e $w$ é chamado de peso,
núcleo, entre outros nomes. No caso das redes convolucionais, o array $w$ contém os
parâmetros da rede convolucional que precisam ser treinados.

A figura a seguir ilustra uma convolução da imagem verde de entrada, utilizando o núcleo da convolução (kernel) em amarelo, deslizando sobre a imagem verde. O resultado da convolução é a imagem rosa, de tamanho ligeiramente menor, devido ao processamento na borda da imagem.

![](http://mourafiq.com/images/posts/convolution_schematic.gif)


In [32]:
# 4-D convolution
def conv4d(x,w,border_mode='same'):
    assert x.ndim == 4
    assert w.ndim == 4
    nb_filters,wd,wh,ww = w.shape
    samples,channels,xh,xw = x.shape
    model = Sequential([
    Convolution2D(nb_filters, wh, ww, 
                  input_shape=(channels, xh, xw),
                  weights=[W], 
                  border_mode=border_mode, bias=False, dim_ordering='th')
    ])
    y = model.predict(x,batch_size=1)
    return y


In [33]:
np.set_printoptions(suppress=True, precision=3)

x = np.zeros((5,7))
x[2,3]=1
x = x.reshape(1,1,5,7)
W = np.array([[1.,2.,3.],
              [4.,5.,6.]]).reshape(1,1,2,3)

y = conv4d(x,W)

print 'x:', x.shape
print x
print 'y:', y.shape
print y
print 'kernel'
w = model.layers[0].get_weights()
print w



x: (1, 1, 5, 7)
[[[[ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  1.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]]]]
y: (1, 1, 5, 7)
[[[[ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  1.  2.  3.  0.  0.]
   [ 0.  0.  4.  5.  6.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]]]]
kernel
[array([[[[ 1.,  2.,  3.],
         [ 4.,  5.,  6.]]]], dtype=float32)]


## Reproduzindo o exemplo da figura animada acima

In [29]:
W = np.array([[1,0,1],
              [0,1,0],
              [1,0,1]]).reshape(1,1,3,3)
x = np.array([[1,1,1,0,0],
              [0,1,1,1,0],
              [0,0,1,1,1],
              [0,0,1,1,0],
              [0,1,1,0,0]]).reshape(1,1,5,5)
y = conv4d(x,W,'valid')
print 'x:', x.shape
print x
print 'y:', y.shape
print y


x: (1, 1, 5, 5)
[[[[1 1 1 0 0]
   [0 1 1 1 0]
   [0 0 1 1 1]
   [0 0 1 1 0]
   [0 1 1 0 0]]]]
y: (1, 1, 3, 3)
[[[[ 4.  3.  4.]
   [ 2.  4.  3.]
   [ 2.  3.  4.]]]]


## 4 amostras de imagem cinza 3x5, 1 filtro com kernel de 3x3 

Neste caso, 
- shape da entrada, x:(4,1,3,5)
- shape dos pesos W:(1, 1, 3,3). 
- shape de saída será y:(4,1,3,5)

In [38]:
W = np.array([[1,0,1],
              [0,1,0],
              [1,0,1]]).reshape(1,1,3,3)
# criando 4 amostras de imagens
x = np.array([[0,0,0,0,0],
              [0,0,1,0,0],
              [0,0,0,0,0]]).reshape(1,1,3,5) * np.arange(4).reshape(4,1,1,1)
y = conv4d(x,W,'same')
print 'x:', x.shape
print x
print 'y:', y.shape
print y


x: (4, 1, 3, 5)
[[[[0 0 0 0 0]
   [0 0 0 0 0]
   [0 0 0 0 0]]]


 [[[0 0 0 0 0]
   [0 0 1 0 0]
   [0 0 0 0 0]]]


 [[[0 0 0 0 0]
   [0 0 2 0 0]
   [0 0 0 0 0]]]


 [[[0 0 0 0 0]
   [0 0 3 0 0]
   [0 0 0 0 0]]]]
y: (4, 1, 3, 5)
[[[[ 0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.]]]


 [[[ 0.  1.  0.  1.  0.]
   [ 0.  0.  1.  0.  0.]
   [ 0.  1.  0.  1.  0.]]]


 [[[ 0.  2.  0.  2.  0.]
   [ 0.  0.  2.  0.  0.]
   [ 0.  2.  0.  2.  0.]]]


 [[[ 0.  3.  0.  3.  0.]
   [ 0.  0.  3.  0.  0.]
   [ 0.  3.  0.  3.  0.]]]]


## 2 filtros com kernel de 3x3, 1 amostra de imagem cinza 3x5

Neste caso, 
- shape da entrada, x:(1,1,3,5)
- shape dos pesos W:(2, 1, 3,3). 
- shape de saída será y:(1,2,3,5)


In [39]:
W = np.array([[[1,0,1],
               [0,1,0],
               [1,0,1]],
              [[0,1,0],
               [1,2,1],
               [0,1,0]]]).reshape(2,1,3,3)
x = np.array([[0,0,0,0,0],
              [0,0,1,0,0],
              [0,0,0,0,0]]).reshape(1,1,3,5)
y = conv4d(x,W,'same')
print 'x:', x.shape
print x
print 'y:', y.shape
print y


x: (1, 1, 3, 5)
[[[[0 0 0 0 0]
   [0 0 1 0 0]
   [0 0 0 0 0]]]]
y: (1, 2, 3, 5)
[[[[ 0.  1.  0.  1.  0.]
   [ 0.  0.  1.  0.  0.]
   [ 0.  1.  0.  1.  0.]]

  [[ 0.  0.  1.  0.  0.]
   [ 0.  1.  2.  1.  0.]
   [ 0.  0.  1.  0.  0.]]]]


## 1 filtro com kernel de 3x3, 1 amostra de imagem 3x5 RGB (3 canais)
Neste caso, 
- shape da entrada, x:(1,3,3,5)
- shape dos pesos W:(1, 1, 3,3). 
- shape de saída será y:(1,2,3,5)


In [45]:
W = np.array([[1,0,1],
              [0,1,0],
              [1,0,1]]).reshape(1,1,3,3)
x = np.array([[0,0,0,0,0],
              [0,0,1,0,0],
              [0,0,0,0,0]]).reshape(1,1,3,5) * np.arange(1,4).reshape(1,3,1,1)
y = conv4d(x,W,'same')
print 'x:', x.shape
print x
print 'y:', y.shape
print y


Exception: Layer weight shape (1, 3, 3, 3) not compatible with provided weight shape (1, 1, 3, 3)

## Exercícios

### Teórico

1. Supondo uma imagem de entrada 100 x 100 e um kernel 3 x 3. Utilizando-se o border_mode como válido, a imagem de saída será de 99 x 99. Se fosse utilizar uma rede densa, com entrada de 100 x 100 e saída 99 x 99, quantos parâmetros seriam necessários para ser treinados? E no caso da rede convolucional? Qual é o fator de redução?

### Práticos

1. Trocar o parâmetro ``border_mode`` para ``same`` e veja a diferença.
2. Rodar a convolução para uma imagem de cinza
3. Rodar a convolução para uma imagem colorida RGB
4. Rodar a convolução para gerar várias bandas de saída como nas redes convolucionais
5. Comparar o tempo de execução desta convolução com outras implementações: openCV por exemplo